# Lab 3 — clustering

In [100]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


from itertools import combinations

%matplotlib inline
plt.style.use("ggplot")

In [137]:
from bokeh.io import show, output_notebook, gridplot
from bokeh.models import Div
from bokeh.layouts import column
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()

Loading BokehJS ...

## 3.11 Clustering Tags

Load the data and prepare the points

In [3]:
with open('data.txt', 'r') as source_file:
    data = json.loads(json.load(source_file)) # Why is this neeeded?

In [25]:
tags = [tag for tag in data]
coordinates = np.array([data[tag] for tag in tags])

In [121]:
centered = coordinates - np.mean(coordinates, axis=0)
cov = (centered.T @ centered) / centered.shape[0]
inc_vals, inv_vecs = np.linalg.eigh(cov)

### Notes for the team: project on 2d with pca and then kmeans or kmeans on 5 components and then use pca just to display?

In [149]:
colors = {
    0: '#ff0000',
    1: '#00ff00',
    2: '#0000ff',
    3: '#ffff00',
    4: '#551a8b'
}
def visualize(k, coords2d):
    clus = KMeans(n_clusters=k).fit(coords2d)

    source = ColumnDataSource(
        data = {
            'x' : [x[0] for x in coords2d],
            'y' : [x[1] for x in coords2d],
            'color' : [colors[x] for x in clus.labels_],
            'tagname' : tags
        }    
    )

    centroids = ColumnDataSource(
        data = {
            'x' : [x[0] for x in clus.cluster_centers_],
            'y' : [x[1] for x in clus.cluster_centers_]
        }
    )

    hover = HoverTool(
            tooltips=[('Tag name', '@tagname')]
    )
    tools = [ResetTool(), PanTool(), WheelZoomTool(), SaveTool(), hover]
    p = figure(tools=tools, width=400, height=400, title='K = '+ str(k))
    p.circle('x', 'y', source=source, color='color')
    p.asterisk('x', 'y', size=30,source=centroids, color='#000000')
    
    return p

For each possible combination of length 2 of eingenvectors (donc principal directions) we plot the clustered data using, for each combination, $k\in \{2,3,4,5\}$

In [151]:
#Try projecting on on different principal directions
for i, j in combinations(range(5), 2):
    xy = inv_vecs[:, [i, j]]
    c = coordinates @ xy
    p = gridplot([[visualize(2, c), visualize(3, c)], [visualize(4, c), visualize(5, c)]])
    title = 'Using components associated to eigenvalues number' + str((i, j))
    #show(p, notebook_handle=True)
    show(column(Div(text=title), p))